Делаем только для аннотации, поэтому используем scmap-cluster, берем из атласа и херачим на эксперимент

In [2]:
library(SingleCellExperiment)
library(scmap)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, 

In [56]:
atlas = read.delim("/home/churkreev_group/vyatkin/data/E_65_85/EmbryoTimecourse2018/download/atlas_E_65/counts_E_65.tsv")
atlas[1:5,1:5]

,cell_1,cell_2,cell_3,cell_4,cell_5
Xkr4,0,0,0.000000,0,0
Gm1992,0,0,0.000000,0,0
Gm37381,0,0,0.000000,0,0
Rp1,0,0,0.000000,0,0
Sox17,0,0,3.302669,0,0


In [57]:
annotation = read.delim("/home/churkreev_group/vyatkin/data/E_65_85/EmbryoTimecourse2018/download/atlas/meta.tab")
annotation = annotation[c('cell', 'celltype')]

cell,celltype
cell_1,Epiblast
cell_2,Primitive Streak
cell_3,NA
cell_4,NA
cell_5,ExE ectoderm
cell_6,Epiblast


In [64]:
annotation = annotation[annotation$cell %in% colnames(atlas), ]

In [91]:
sce_atlas <- SingleCellExperiment(assays = list(logcounts = as.matrix(atlas)), colData = annotation)

# use gene names as feature symbols
rowData(sce_atlas)$feature_symbol <- rownames(sce_atlas)
# remove features with duplicated names
sce_atlas <- sce_atlas[!duplicated(rownames(sce_atlas)), ]

In [114]:
sce_atlas <- selectFeatures(sce_atlas, suppress_plot = TRUE)

Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”

In [93]:
table(rowData(sce_atlas)$scmap_features)


FALSE  TRUE 
28952   500 

# functions

In [9]:
prepare_annotation <- function(atlas_annotation_path, atlas){
    annotation = read.delim(atlas_annotation_path)
    annotation = annotation[c('cell', 'celltype')]
    annotation = annotation[annotation$cell %in% colnames(atlas), ]
    
    return(annotation)
}


prepare_atlas <- function(atlas_path, atlas_annotation_path){
    atlas = read.delim(atlas_path)
   
    sce_atlas <- SingleCellExperiment(assays = list(logcounts = as.matrix(atlas)), colData = prepare_annotation(atlas_annotation_path, atlas))

    # use gene names as feature symbols
    rowData(sce_atlas)$feature_symbol <- rownames(sce_atlas)
    # remove features with duplicated names
    sce_atlas <- sce_atlas[!duplicated(rownames(sce_atlas)), ]
    sce_atlas <- selectFeatures(sce_atlas, suppress_plot = TRUE)
    
    sce_atlas <- indexCluster(sce_atlas, cluster_col = "celltype")
    
    return(sce_atlas)
}


prepare_experiment <- function(experiment_path, separator){
    experiment = read.delim(experiment_path, row.names=1, sep=separator)
    nams = sub("_.*","",rownames(experiment))
    rownames(experiment) = make.names(nams,unique = T)
    
    sce_experiment <- SingleCellExperiment(assays = list(logcounts = as.matrix(experiment)))

    # use gene names as feature symbols
    rowData(sce_experiment)$feature_symbol <- rownames(sce_experiment)
    # remove features with duplicated names
    sce_experiment <- sce_experiment[!duplicated(rownames(sce_experiment)), ]
    sce_experiment <- selectFeatures(sce_experiment, suppress_plot = TRUE)
    
    return(sce_experiment)
}

In [121]:
sce_atlas = prepare_atlas("/home/churkreev_group/vyatkin/data/E_65_85/EmbryoTimecourse2018/download/atlas_E_65/counts_E_65.tsv",
             "/home/churkreev_group/vyatkin/data/E_65_85/EmbryoTimecourse2018/download/atlas/meta.tab")

sce_experiment = prepare_experiment("/home/churkreev_group/data_E3.5_to_E6.5/GSE100597_count_table_QC_filtered.txt")

scmapCluster_results <- scmapCluster(
  projection = sce_experiment, 
  index_list = list(
    sce_atlas = metadata(sce_atlas)$scmap_cluster_index
  )
)

Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”Warning message in setFeatures(projection, rownames(index)):
“Features 1010001N08Rik, 1700001D01Rik, 1700007F19Rik, 1700007K09Rik, 1700013F07Rik, 1700029E06Rik, 1700057H15Rik, 2210011C24Rik, 2610020F03Rik, 3830432H09Rik, 4922502D21Rik, 4930434B07Rik, 4930477E14Rik, 4930505N22Rik, 4930533D04Rik, 4930538E20Rik, 4930558N11Rik, 4930578M01Rik, 4933427E11Rik, 4933431M02Rik, 5830416I19Rik, 6030466F02Rik, 6330403N20Rik, 9330175M20Rik, AC102435.3, AC113508.1, AC131684.1, AC131777.2, AC132085.1, AC133083.2, AC149586.1, AC153556.3, AC155709.2, AC160121.1, AC160146.1, AC160409.1, AC160562.1, Adgb, AI839979, AL603745.3, Amer3, Ankub1, Axdnd1, Bex1, Bex3, Bex4, C1s2, C230079O03Rik, Ccdc182, Cfap58, Chia1, Cit

In [122]:
table(scmapCluster_results$scmap_cluster_labs)


            Epiblast         ExE ectoderm         ExE endoderm 
                  26                    4                   21 
                  NA    Parietal endoderm     Primitive Streak 
                   7                   35                   62 
Rostral neurectoderm           unassigned    Visceral endoderm 
                  35                  498                   33 

# 6_5

In [21]:
sce_atlas = prepare_atlas("/home/churkreev_group/chukreev/batch_effect/atlas_6_5_corrected.tsv",
             "/home/churkreev_group/vyatkin/data/E_65_85/EmbryoTimecourse2018/download/atlas/meta.tab")


Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”

In [22]:
sce_experiment = prepare_experiment("/home/churkreev_group/chukreev/batch_effect/experiment_6_5_corrected.tsv", separator = '\t')

Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”

In [23]:
scmapCluster_results <- scmapCluster(
  projection = sce_experiment, 
  index_list = list(
    sce_atlas = metadata(sce_atlas)$scmap_cluster_index
  )
)

In [24]:
table(scmapCluster_results$scmap_cluster_labs)


     ExE ectoderm      ExE endoderm Visceral endoderm 
              127                 1                39 

In [32]:
write.csv(scmapCluster_results$scmap_cluster_labs, file='/home/churkreev_group/chukreev/batch_effect/cluster_annotation.txt')

# 7_5

In [16]:
sce_atlas = prepare_atlas("/home/churkreev_group/chukreev/batch_effect/atlas_725_corrected.tsv",
             "/home/churkreev_group/vyatkin/data/E_65_85/EmbryoTimecourse2018/download/atlas/meta.tab")


Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”

In [19]:
sce_experiment = prepare_experiment("/home/churkreev_group/chukreev/batch_effect/experiment_725_corrected.tsv_imputed", separator = '\t')



Warning message in linearModel(object, n_features):
“Your object does not contain counts() slot. Dropouts were calculated using logcounts() slot...”

ERROR: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...): 0 (non-NA) cases


In [ ]:
scmapCluster_results <- scmapCluster(
  projection = sce_experiment, 
  index_list = list(
    sce_atlas = metadata(sce_atlas)$scmap_cluster_index
  )
)
table(scmapCluster_results$scmap_cluster_labs)

# 6_5 леша